In [ ]:
# imports 
import numpy as np
import matplotlib.pyplot as plt
import csv
import networkx as nx

from scipy.linalg import fractional_matrix_power
from sklearn.cluster import KMeans

# typing 
from numpy import ndarray

def load_data(filename: str) -> list[tuple[int, int]]:
        '''Load data from file and return it as a list of frozensets.'''
        with open(f'data/{filename}.dat', newline='') as csvfile:
            edges_reader = list(csv.reader(csvfile, delimiter=',', quotechar='|'))
            values = len(edges_reader[0])
            if values==2:
                edges = list(tuple((int(a), int(b))) for a, b in edges_reader)
            else:
                edges = list(tuple((int(a), int(b))) for a, b, c in edges_reader)
        return edges
    
def get_adj_mtx(edges_list: list[tuple[int, int]]) -> ndarray:
    max_idx = max(max(*edges_list))
    A = np.zeros(shape=(max_idx, max_idx), dtype=int)
    for i, j in edges_list:
        A[i-1, j-1] = 1
        A[j-1, i-1] = 1
    return A

def draw_graph(A: ndarray, labels: ndarray):
    graph = nx.from_numpy_array(A)
    pos = nx.spring_layout(graph)
    nx.draw_networkx(graph, pos=pos, node_size=10, node_color=labels, with_labels=False, cmap=plt.cm.Set3)  # type: ignore
    plt.show()

def spectral_clustering(A: ndarray, k: int) -> ndarray:
    D = np.diag(np.sum(A, axis=1))
    D_inv = np.linalg.inv(np.sqrt(D))
    L = D_inv @ A @ D_inv

    # get eigenvalues and eigenvectors
    _, eigenvectors = np.linalg.eigh(L)

    # exctract the k biggest eigenvectors in X
    X = eigenvectors[:, -k:]
    
    # renormalize X rows to get Y
    Y = X / np.power(np.sum(X**2, axis=1, keepdims=True), 0.5)
    
    # cluster the rows of Y using k-means clustering
    kmeans = KMeans(n_clusters=k).fit(Y)
    labels = kmeans.labels_
    return labels  # type: ignore

def plot_fiedler(A: ndarray):
    # get eigenvalues and eigenvectors
    _, eigenvectors = np.linalg.eigh(A)

    # extract the Fiedler vector (second smallest eigenvector)
    fiedler_eigenvector = sorted(eigenvectors[:, 1])
    plt.plot(np.arange(len(fiedler_eigenvector)), fiedler_eigenvector)
    plt.show()


# Fiedler vector example

In [ ]:
# graphs: example1, example2
graph = load_data('example2')
k = 2

A = get_adj_mtx(graph)

plt.spy(A)
plt.show()
plot_fiedler(A)
l = spectral_clustering(A, k)
draw_graph(A, labels=l)
